In [ ]:
from pyspark.mllib.recommendation import ALS, Rating
from pyspark import SparkContext

sc = SparkContext(appName="MovieRecommender")
artistData = sc.textFile("s3://aws-logs-135040824471-us-west-2/elasticmapreduce/audio-data/artist_data.txt")
artistAlias = sc.textFile("s3://aws-logs-135040824471-us-west-2/elasticmapreduce/audio-data/artist_alias.txt")
userArtistData = sc.textFile("s3://aws-logs-135040824471-us-west-2/elasticmapreduce/audio-data/user_artist_data.txt")

artist_data = artistData.collect()
artist_alias = artistAlias.collect()
user_artist_data = userArtistData.collect()

#compute the mapping between inaccurate artist id and canonical artist id
artist_id_dict = {}
for line in artist_alias:
    artist = line.split("\t")
    artist_id_dict[artist[0]] = artist[1]

#compute the mapping between artist id and artist name
artist_name_dict = {}
for line in artist_data:
    artist = line.split("\t")
    if len(artist) >= 2:
        artist_name_dict[artist[0]] = artist[1]

#transform artist id in userArtistData into canonical ones
def correctArtistId(line):
    line = line.split(" ")
    
    if line[1] in artist_id_dict:
        return Rating(int(line[0]),int(artist_id_dict[line[1]]),int(line[2]))
    else:
        return Rating(int(line[0]),int(line[1]),int(line[2]))
          
userArtistData = userArtistData.map(correctArtistId)

model = ALS.trainImplicit(userArtistData, rank = 10, iterations = 5, lambda_ = 0.01, alpha = 1.0)
ratings = model.recommendProducts(2093760, 10)

print "Top 10 recommendations for user 2093760:"
for i in range(0,10):
    print "Artist " + str(i+1) + " : " + str(artist_name_dict[str(ratings[i].product)])